In [16]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from ape import networks, Contract
import pandas as pd
import altair as alt

from helpers.coingecko import get_votium_prices
from helpers.votium import CVX_FEE
from helpers.formatter import dollar_format

In [ ]:
networks.parse_network_choice("ethereum:mainnet").__enter__()

In [18]:
_, _, badger_price, fxs_price = get_votium_prices()
data = [["${:.2f}".format(badger_price), "${:.2f}".format(fxs_price)]]
headers = ["BADGER price", "FXS price"]
df_prices = pd.DataFrame(data, columns=headers)
df_prices

,BADGER price,FXS price
0,$2.80,$10.13


In [19]:
# Incentive program. Starts on 9th Jan, last incentive post on the 26th Jan
# first incentive tx: https://etherscan.io/tx/0x6f89da605f5cb96ffab69a2abab83532c57890e41d30dcb36d16e303075b637d
# apr gets actioned in: https://facts.frax.finance/gauges/cycles/2023-C54 (12 Jan 2023 - 19 Jan 2023)
FIRST_BLOCK_APR_INFLUENCED = 16386999
LAST_BLOCK_APR_INFLUENCED = 16587476
WEEKLY = 1500
TOTAL_INCENTIVES = 1500 * 4

In [20]:
# FXS claims & pendants. Directly querying from farm to avoid estimations based on pool capture etc
# There was already a claim on the 3rd Jan, so 3rd Jan till the round of the 9th Jan is considered un-incentivesed
farm = Contract("0x5a92EF27f4baA7C766aee6d751f754EBdEBd9fae")
dao_private_vault = "0xa895B89D74a6BC23a284a0526e123ea776674cF5"
FIRST_CLAIM_BLOCK = 16478637  # https://etherscan.io/tx/0x6b08c3763f4d38993a0eaca06558184549eeab55f5c51e71ee774e44955a7107
SECOND_CLAIM_BLOCK = 16529630 # https://etherscan.io/tx/0xe37313ed6878cc57557c1c3196e51c28551de39ec21d9b3522d6dbb4183c7223
THIRD_CLAIM_BLOCK = 16571621  # https://etherscan.io/tx/0x4941e62742158e7efe1efabdc6a346ac1be8bad526b07ffe3fffde9689eb3b62
fxs_earned_unincentived = farm.earned(dao_private_vault, block_identifier=FIRST_BLOCK_APR_INFLUENCED)[0] / 1e18
fxs_earned = 0
for block in [FIRST_CLAIM_BLOCK, SECOND_CLAIM_BLOCK, THIRD_CLAIM_BLOCK]:
    fxs_earned
    fxs_earned += farm.earned(dao_private_vault, block_identifier=block)[0] / 1e18
fxs_pendant = farm.earned(dao_private_vault, block_identifier=LAST_BLOCK_APR_INFLUENCED)[0] / 1e18
total_fxs_earned = ((fxs_earned - fxs_earned_unincentived) + fxs_pendant) * (1 - CVX_FEE)

In [21]:
# Cost vs earnings in dollar terms
earnings = total_fxs_earned * fxs_price
costs = TOTAL_INCENTIVES * badger_price
data = [[dollar_format(costs), dollar_format(earnings)]]
headers = ["Treasury Decision Cost", "Earnings"]
df_cost_vs_earning = pd.DataFrame(data, columns=headers)
df_cost_vs_earning

,Treasury Decision Cost,Earnings
0,$16800.0000,$12383.4538


In [22]:
# Basic groupe bar chart
df = pd.DataFrame([[earnings, 'R'], 
                   [costs, 'C']],
                  columns=['$ Value', 'P&L'])

chart = alt.Chart(df, width=400).mark_bar().encode(
   x=alt.X('P&L'),
   y=alt.Y('$ Value'),
   color=alt.Color('P&L', scale=alt.Scale(range=['#FF5733', '#33FF47']))
)

chart.display() 

alt.Chart(...)